In [58]:
from langchain_community.tools import DuckDuckGoSearchRun
search = DuckDuckGoSearchRun()

In [59]:
search

DuckDuckGoSearchRun()

In [60]:

from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from operator import itemgetter
from dotenv import load_dotenv
load_dotenv('/Users/eshantdas/Desktop/Youtube/Youtube_GenAI_Test_code/myenv/.env')
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq

model = ChatGroq(
    temperature=0,
    model="llama3-70b-8192",
)

In [61]:
from langchain_community.tools import YouTubeSearchTool
youtube = YouTubeSearchTool()


In [62]:
youtube

YouTubeSearchTool()

In [63]:
from langchain import hub
from langchain.agents import AgentExecutor, create_tool_calling_agent

prompt = hub.pull("hwchase17/openai-tools-agent")
prompt.pretty_print()

================================ System Message ================================

You are a helpful assistant

============================= Messages Placeholder =============================

{chat_history}

================================ Human Message =================================

{input}

============================= Messages Placeholder =============================

{agent_scratchpad}


In [64]:
tools=[search,youtube]

In [65]:
# Construct the tool calling agent
agent = create_tool_calling_agent(model, tools, prompt)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,return_intermediate_steps=True,handle_parsing_errors=True)


In [66]:
a=agent_executor.invoke(
    {
        "input": "Do you know about Krish naik youtube channel"
    }
)



> Entering new AgentExecutor chain...

Invoking: `youtube_search` with `Krish Naik`


['https://www.youtube.com/watch?v=HEHUpBO8UVc&pp=ygUKS3Jpc2ggTmFpaw%3D%3D', 'https://www.youtube.com/watch?v=3SRh2nzN2DM&pp=ygUKS3Jpc2ggTmFpaw%3D%3D']It seems like the tool call yielded some YouTube video links related to Krish Naik's channel. Would you like me to provide more information about Krish Naik or his YouTube channel?

> Finished chain.


In [67]:
a

{'input': 'Do you know about Krish naik youtube channel',
 'output': "It seems like the tool call yielded some YouTube video links related to Krish Naik's channel. Would you like me to provide more information about Krish Naik or his YouTube channel?",
 'intermediate_steps': [(ToolAgentAction(tool='youtube_search', tool_input='Krish Naik', log='\nInvoking: `youtube_search` with `Krish Naik`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'id': 'call_5vxd', 'function': {'arguments': '{"__arg1":"Krish Naik"}', 'name': 'youtube_search'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None}, id='run-4e9811e2-6d18-4a54-9ff7-c9bcb13736ca', tool_calls=[{'name': 'youtube_search', 'args': {'__arg1': 'Krish Naik'}, 'id': 'call_5vxd', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1103, 'output_tokens': 48, 'total_tokens': 1151}, tool_call_chunks=[{'name': 'youtube_search', 'args': '{"__arg1":"Krish Naik"}', 'id': 'c

In [68]:
a=agent_executor.invoke(
    {
        "input": "SEarch about Krish naik in duckduck go website"
    }
)



> Entering new AgentExecutor chain...

Invoking: `duckduckgo_search` with `{'query': 'Krish naik'}`


Krish Naik is a data science educator and creator who offers free videos and materials to learn generative AI (LLM's) and other topics. Watch his roadmap, playlists, projects and live sessions on his YouTube channel. Langchain Complete Playlist: https://www.youtube.com/playlist?list=PLZoTAELRMXVORE4VF7WQ_fAl0L1Gljtar-----... Krish naik teaches surface level stuff. I didn't know him. He is a copy paste person, doesn't know the tech behind , only trying to make videos. I've been watching his videos for a while now and being a beginner, I assumed he was pretty good. However, I've seen a few people criticise him for…. Complete Python Guide: https://github.com/krishnaik06/The-Grand-Complete-Data-Science-Materials/tree/main-----... Krish C Naik is a cofounder of iNeuron and a machine learning educator and mentor. He has a YouTube channel where he explains various topics on ML, AI, and data

In [69]:
a

{'input': 'SEarch about Krish naik in duckduck go website',
 'output': 'It seems like the tool has provided some YouTube video links related to Krish Naik. Would you like me to provide a summary of what Krish Naik is about, based on the previous tool call results?',
 'intermediate_steps': [(ToolAgentAction(tool='duckduckgo_search', tool_input={'query': 'Krish naik'}, log="\nInvoking: `duckduckgo_search` with `{'query': 'Krish naik'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'id': 'call_8a4h', 'function': {'arguments': '{"query":"Krish naik"}', 'name': 'duckduckgo_search'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None}, id='run-6dcc8dae-956c-456c-a815-ee0f99d82ec8', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'Krish naik'}, 'id': 'call_8a4h', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1105, 'output_tokens': 48, 'total_tokens': 1153}, tool_call_chunks=[{'name': 'duckduckgo_s

In [70]:
from langchain.tools import BaseTool, StructuredTool, tool

@tool
def llm_call(query: str) -> str:
    """If any complicated ligical reasoning related query comes then use llm_call"""
    prompt = ChatPromptTemplate.from_template("{content}")
    output_parser = StrOutputParser()

    chain = prompt | model | output_parser

    answer=chain.invoke({"content": query})
    return answer

tools.append(llm_call)
    

In [71]:
tools

[DuckDuckGoSearchRun(),
 YouTubeSearchTool(),
 StructuredTool(name='llm_call', description='If any complicated ligical reasoning related query comes then use llm_call', args_schema=<class 'pydantic.v1.main.llm_callSchema'>, func=<function llm_call at 0x167056660>)]

In [72]:
# Construct the tool calling agent
agent = create_tool_calling_agent(model, tools, prompt)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,return_intermediate_steps=True,handle_parsing_errors=True)


In [73]:
a=agent_executor.invoke(
    {
        "input": "What is 30 + 40"
    }
)



> Entering new AgentExecutor chain...
<tool-use>{"tool_calls": []}</tool-use>

> Finished chain.


In [74]:
a

{'input': 'What is 30 + 40',
 'output': '<tool-use>{"tool_calls": []}</tool-use>',
 'intermediate_steps': []}

In [75]:
a=agent_executor.invoke(
    {
        "input": "Tell me some youtube channels to find content about Rohit Sharma"
    }
)



> Entering new AgentExecutor chain...

Invoking: `youtube_search` with `Rohit Sharma`


['https://www.youtube.com/watch?v=TENNdaKXmP4&pp=ygUMUm9oaXQgU2hhcm1h', 'https://www.youtube.com/watch?v=S3cY5HM0d_8&pp=ygUMUm9oaXQgU2hhcm1h']You can watch those videos to find content about Rohit Sharma.

> Finished chain.


In [79]:
a

{'input': 'Tell me some youtube channels to find content about Rohit Sharma',
 'output': 'You can watch those videos to find content about Rohit Sharma.',
 'intermediate_steps': [(ToolAgentAction(tool='youtube_search', tool_input='Rohit Sharma', log='\nInvoking: `youtube_search` with `Rohit Sharma`\n\n\n', message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'id': 'call_rh8m', 'function': {'arguments': '{"__arg1":"Rohit Sharma"}', 'name': 'youtube_search'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls', 'logprobs': None}, id='run-1907aac6-5913-4202-8a94-58a4766956e2', tool_calls=[{'name': 'youtube_search', 'args': {'__arg1': 'Rohit Sharma'}, 'id': 'call_rh8m', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1210, 'output_tokens': 48, 'total_tokens': 1258}, tool_call_chunks=[{'name': 'youtube_search', 'args': '{"__arg1":"Rohit Sharma"}', 'id': 'call_rh8m', 'index': None, 'type': 'tool_call_chunk'}])], tool_call_id='call_rh8m'),
   "

In [90]:
@tool
def history_management(query: str) -> str:
    """This tool looks into the history and answers the question"""
    prompt = ChatPromptTemplate.from_template("{content}")
    output_parser = StrOutputParser()

    chain = prompt | model | output_parser

    answer=chain.invoke({"content": f"This is the query : {query}. Look into the history and give the answer"})
    return answer 

In [91]:
tools.append(history_management)

In [92]:
tools

[DuckDuckGoSearchRun(),
 YouTubeSearchTool(),
 StructuredTool(name='llm_call', description='If any complicated ligical reasoning related query comes then use llm_call', args_schema=<class 'pydantic.v1.main.llm_callSchema'>, func=<function llm_call at 0x167056660>),
 StructuredTool(name='history_management', description='This tool looks into the history and answers the question', args_schema=<class 'pydantic.v1.main.history_managementSchema'>, func=<function history_management at 0x167056b60>)]

In [93]:
from langchain.memory import ConversationBufferWindowMemory
memory = ConversationBufferWindowMemory( k=3)

In [94]:
# Construct the tool calling agent
agent = create_tool_calling_agent(model, tools, prompt)
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True,handle_parsing_errors=True,memory=memory)


In [95]:
a=agent_executor.invoke(
    {
        "input": "My pet name is Theo"
    }
)



> Entering new AgentExecutor chain...

Invoking: `history_management` with `{'query': 'My pet name is Theo'}`


Nice to meet you, Theo!

The origin and history of the name Theo are fascinating. Theo is a shortened form of Theodoros, a Greek name that means "gift of God" or "divine gift." Theodoros is derived from the Greek words "theos," meaning "god," and "doron," meaning "gift."

In ancient Greece, Theodoros was a common name, and it was often given to children born into families of high social status. The name was also associated with several notable figures in Greek mythology and history, including Theodoros of Samos, a 6th-century BC engineer and inventor.

During the Middle Ages, the name Theodoros was adopted by early Christians, who saw it as a way to express their devotion to God. The name became popular in Europe during the 17th and 18th centuries, particularly in Germany and England.

In modern times, the shortened form Theo has become increasingly popular as a given name.

In [96]:
a

{'input': 'My pet name is Theo',
 'history': '',
 'output': "Nice to meet you, Theo! It seems like your name has a lot of interesting facts and meanings behind it. I'm glad I could help you learn more about your name."}

In [97]:
a['output']

"Nice to meet you, Theo! It seems like your name has a lot of interesting facts and meanings behind it. I'm glad I could help you learn more about your name."

In [98]:
a=agent_executor.invoke(
    {
        "input": "What is my pet name"
    }
)



> Entering new AgentExecutor chain...

Invoking: `llm_call` with `{'query': 'What is my pet name'}`


I'm happy to help! However, I don't have any information about you or your personal relationships, so I don't know what your pet name might be.

A pet name is a special nickname that is often given to someone by a loved one, such as a partner, family member, or close friend. It's usually a term of endearment that is unique to that person and is used to express affection or closeness.

If you're wondering what your pet name might be, you might consider asking the person who gave you the nickname!I see that the tool didn't provide a direct answer. In that case, I'll respond directly without using a tool. It seems that you don't have a pet name that I'm aware of, and as the tool suggested, you might want to ask someone close to you if they have given you a special nickname.

> Finished chain.


In [99]:
a

{'input': 'What is my pet name',
 'history': "Human: My pet name is Theo\nAI: Nice to meet you, Theo! It seems like your name has a lot of interesting facts and meanings behind it. I'm glad I could help you learn more about your name.",
 'output': "I see that the tool didn't provide a direct answer. In that case, I'll respond directly without using a tool. It seems that you don't have a pet name that I'm aware of, and as the tool suggested, you might want to ask someone close to you if they have given you a special nickname."}

In [43]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain.memory import ChatMessageHistory

prompt = hub.pull("hwchase17/react")

memory = ChatMessageHistory(session_id="test-session")

In [44]:
from langchain_core.runnables.history import RunnableWithMessageHistory


llm = model
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)

agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    # This is needed because in most real world scenarios, a session id is needed
    # It isn't really used here because we are using a simple in memory ChatMessageHistory
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [49]:
agent_with_chat_history.invoke(
    {"input": "What is my pets name"},
    config={"configurable": {"session_id": "123"}},
)

{'input': 'What is my pets name',
 'chat_history': [HumanMessage(content='How many people live in canada?'),
  AIMessage(content='As of April 1, 2024, the population of Canada is approximately 41,012,563.'),
  HumanMessage(content='My name is Eshant?'),
  AIMessage(content='The question "My name is Eshant?" is likely a playful reference to the famous movie quote from "Taken" and is not a serious inquiry about the person\'s name.'),
  HumanMessage(content='What is my name'),
  AIMessage(content="I'm sorry, but I don't have enough information to determine your name."),
  HumanMessage(content='My pets mame is Theo'),
  AIMessage(content='The name "Theo" is of Greek origin and means "Gift Of God" or "Divine Gift".')],
 'output': "I'm sorry, but I don't have enough information to answer this question. Please provide more context or details about your pet, and I'll do my best to help!"}